In [3]:
import openai
import os
openai.api_key=os.environ['OPENAI_API_KEY']
openai.base_url=os.environ['BASE_URL']

# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt,model="gpt-3.5-turbo"):
    messages = [{"role":"user","content":prompt}]
    response = openai.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0,
    )
    return response.choices[0].message.content

In [9]:
# 1 原则一 编写清晰、具体的指令

# 1.1 使用分隔符清晰地表示输入的不同部分
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

提供清晰、具体的指示可以引导模型朝向所需输出，降低收到无关或不正确响应的可能性，不要与简短的提示混淆，更长的提示可以提供更多清晰度和上下文信息。


In [7]:
# 1.2 寻求结构化的输出
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

```json
[
    {
        "book_id": 1,
        "title": "The Night Circus",
        "author": "Erin Morgenstern",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "1984",
        "author": "George Orwell",
        "genre": "Dystopian"
    },
    {
        "book_id": 3,
        "title": "Pride and Prejudice",
        "author": "Jane Austen",
        "genre": "Romance"
    }
]
```


In [8]:
# 1.3 要求模型检查是否满足条件

# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水倒入水壶中，并将水壶放在火炉上烧开。
第二步 - 取出一个茶杯，将茶包放入杯中。
第三步 - 等待水沸腾后，将热水倒入茶杯中浸泡茶包。
第四步 - 等待几分钟，让茶叶充分浸泡。
第五步 - 使用茶匙取出茶包，并搅拌如果需要加入糖或牛奶进行调味。
第六步 - 您现在可以享受美味的茶了。


In [10]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


In [11]:
# 1.4 提供少量示例（少样本提示词，Few-shot prompting）
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 就像钢铁在火中经过锻炼变得坚固，人也需要经历挑战和困难才能变得坚韧。在困难面前，保持乐观、坚定和不屈的态度，才能展现出真正的韧性。继续努力，坚持不懈，你会发现自己变得越来越强大。


In [12]:
# 2 给模型时间去思考

# 2.1 指定完成任务所需的步骤
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1- 兄妹在山上打水时遭遇意外但仍保持冒险精神。
2- Alors que Jack et Jill se rendaient à un puits au sommet d'une colline dans un charmant village, ils ont eu un accident mais ont continué à explorer joyeusement. 
3- Jack, Jill
4- {
   "French_summary": "Alors que Jack et Jill se rendaient à un puits au sommet d'une colline dans un charmant village, ils ont eu un accident mais ont continué à explorer joyeusement.",
   "num_names": 2
}


In [13]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要: 在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水，他们一路唱歌欢笑，不幸却相继降临。但他们最终安然返回家园，冒险精神不减，继续探索。

翻译: In a charming village, siblings Jack and Jill set off to a well on the mountain top to fetch water. Despite some unfortunate events, they return home safely with their adventurous spirit intact, continuing to explore joyfully.

名称: Jack, Jill

输出 JSON: {"English_summary": "In a charming village, siblings Jack and Jill set off to a well on the mountain top to fetch water. Despite some unfortunate events, they return home safely with their adventurous spirit intact, continuing to explore joyfully.", "num_names": 2}


In [14]:
# 2.2 指导模型在下结论之前找出一个自己的解法
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案正确，首年运营的总费用应为450x + 100,000美元。


In [15]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：
首先，自己解决问题，解决问题时列数学表达式。
然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
**学生的计算结果：学生的计算结果文本
实际计算结果：实际计算结果文本
学生的计算结果和实际计算结果是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否**
学生的成绩：正确或不正确

问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

首年运营总费用为：
土地费用：$100 * x
太阳能电池板费用：$250 * x
维护费用：$100,000 + ($10 * x)

总费用 = 土地费用 + 太阳能电池板费用 + 维护费用
总费用 = $100x + $250x + $100,000 + $10x
总费用 = $360x + $100,000

学生的计算结果：450x+100,000
实际计算结果：360x+100,000
学生的计算结果和实际计算结果是否相同：否
学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确


In [16]:
# 3 局限性 幻觉
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_completion(prompt)
print(response)

Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款智能牙刷，具有超薄设计。该牙刷配备了高级技术，可以有效地清洁牙齿并保持口腔卫生。它还具有智能功能，可以连接到手机应用程序，帮助您跟踪刷牙时间和习惯。AeroGlide UltraSlim Smart Toothbrush还采用耐用的材料制成，可以持久使用。总的来说，这是一款功能强大的牙刷，适合追求高品质口腔护理的人群使用。
